<a href="https://colab.research.google.com/github/Dhananjana97/ML-Ride-Fair/blob/master/XGBoost_change_fareper_dist2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
from google.colab import drive 
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import common Modules

In [141]:
import pandas as pd
import numpy as np
import xlrd


import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score

from datetime import datetime




# FileLoad

In [142]:
train_df=pd.read_csv('/content/drive/My Drive/ML Project/train.csv');
test_df=pd.read_csv('/content/drive/My Drive/ML Project/test.csv');
submission_df=pd.read_csv('/content/drive/My Drive/ML Project/sample_submission.csv')
train_df.drop_duplicates(inplace = True)
test_df.drop_duplicates(inplace = True)
train_df

# train_df = train_df[train_df['duration']!=0]
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,correct


# Seperate label clumn

In [143]:
lbl=[]
for index, row in train_df.iterrows():
  label=row['label']
  
  if(label=="correct"):
    lbl.append(1)
  elif(label=="incorrect"):
    lbl.append(0)
lbl

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,


In [144]:
train_df['lb']=lbl


In [145]:
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,lb
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct,1
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct,1
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct,1
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct,1
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct,1
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct,1
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct,1
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,correct,1


In [146]:
labels_df=pd.DataFrame(train_df['lb'])
features_df=train_df.drop(labels=['label','lb'],axis=1)
features_df

features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31


#Import model related modules

In [147]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn_pandas import CategoricalImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
# from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from pprint import pprint
from sklearn.metrics import fbeta_score
from math import radians, cos, sin, asin, sqrt 

# Calc Distance 

In [148]:


def getDistance(df):
  # The math module contains a function named 
    # radians which converts from degrees to radians. 
  dist=[]
  for index, row in df.iterrows():
    lon1 = radians(row['pick_lon']) 
    lon2 = radians(row['drop_lon']) 
    lat1 = radians(row['pick_lat']) 
    lat2 = radians(row['drop_lat']) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    distance=c * r
    dist.append(distance)
  
  return dist
      
      


In [149]:
getDistance(features_df)

[5.092769918614749,
 3.1680575840028093,
 6.305395096498956,
 0.8619461197905727,
 8.14778220061441,
 24.207039248887845,
 4.77762352223266,
 5.322543700241312,
 1.0353018356743493,
 2.930715230104055,
 14.381001132845066,
 4.515655217602484,
 9.424518630693273,
 1.4822326128545675,
 1.4400702368954967,
 13.134533272021095,
 5.123992648484041,
 1.2646648169662118,
 9.946708664968586,
 2.4989545167381277,
 2.2814676303202774,
 16.783013705293456,
 1.9297374974341521,
 4.2524115423095665,
 8.774419577321087,
 2.158639601316546,
 0.22512869093339952,
 1.5586563952189847,
 5.044182192380528,
 4.332270836375877,
 2.7020831310547613,
 2.4344507883203668,
 6.095000666312774,
 9.932622235546958,
 0.7754995733372814,
 1.1968734616119863,
 1.955358507262967,
 7.221149247721649,
 6.4991273847408655,
 2.7143833765541463,
 10.804860544656814,
 0.7983078488963857,
 1.5689911732292072,
 0.7124683760151019,
 2.4514238318216366,
 1.7361632222406627,
 0.6001069791989461,
 0.01236083468686051,
 5.5632191

In [150]:
features_df['dist']=getDistance(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077


# make bins of pickup time

In [151]:
# # datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
# def get_time_type(df):
#   pickup_slot=[]
#   both="Both"
#   school="School"
#   office="Office"
#   free="free"
#   for index, row in df.iterrows():
#     pickup_datetime=row['pickup_time']
#     both_start="6:00"
#     both_end="9:30"
#     school_start="13:30"
#     school_end="14:30"
#     office_start="16:30"
#     office_end="19:30"

#     pickup_time=datetime.strptime(pickup_datetime.split(" ")[1],'%H:%M')
#     both_start_time=datetime.strptime(both_start,'%H:%M')
#     both_end_time=datetime.strptime(both_end,'%H:%M')
#     school_start_time=datetime.strptime(school_start,'%H:%M')
#     school_end_time=datetime.strptime(school_end,'%H:%M')
#     office_start_time=datetime.strptime(office_start,'%H:%M')
#     office_end_time=datetime.strptime(office_end,'%H:%M')
    
#     if(pickup_time> both_start_time and pickup_time<both_end_time):
#       pickup_slot.append(both)
#       # print(pickup_time,both)
#     elif(pickup_time> school_start_time and pickup_time<school_end_time):
#       pickup_slot.append(school)
#       # print(pickup_time,school)
#     elif(pickup_time>office_start_time and pickup_time<office_end_time):
#       pickup_slot.append(office)
#       # print(pickup_time,office)
#     else:
#       pickup_slot.append(free)
#       # print(pickup_time,free)
    

    
#   return pickup_slot

In [152]:
# features_df['Time_type']=get_time_type(features_df)
# features_df

# fare per distance

In [153]:
import math

def get_fare_per_distance(df):
  fare_per_dist=[]
  for index, row in df.iterrows():
    
    
    if(row['dist']==0):
     
      fare_per_dist.append(0)
      print(index,'0')
    else:
      fare_per_distance=(float(row['fare'])-(float(row['meter_waiting_fare'])+float(row['additional_fare'])))/float(row['dist'])
      # if(math.isnan(fare_per_distance)):
      #   fare_per_distance=0
      # else:
      # fare_per_distance=fare_per_distance
      print(index,fare_per_distance)
      fare_per_dist.append(fare_per_distance)
  
  return fare_per_dist




In [154]:
get_fare_per_distance(features_df)

Streaming output truncated to the last 5000 lines.
12176 60.81742180600344
12177 67.39382134261399
12178 45.38330704088775
12179 101765.64412107253
12180 48.78334937777761
12181 73750.93673465875
12182 47.990648156166905
12183 41.29111100130782
12184 49.21013261799155
12185 57.58719261542081
12186 42.62768973490465
12187 65.36958817988467
12188 40.66755203911348
12189 39.098560107104326
12190 37.577230753328955
12191 22.05001705953534
12192 67.98148444266417
12193 31.675481578863643
12194 101.43690231180352
12195 39.93312243759037
12196 56.2403354026749
12197 38.427755833554976
12198 44.73143381526923
12199 36.664438221279354
12200 44.008831732774304
12201 52.700868168215464
12202 47.05965439801223
12203 52.26556193888909
12204 47.94536996264846
12205 52.542152458845834
12206 42643.76211163702
12207 46.70267772993097
12208 38.96986890289784
12209 53.15906935147552
12210 39.16221864113386
12211 85.10590208510014
12212 60.856253541097594
12213 37.58487054088617
12214 55.75038478309266
12

[51.01742355379603,
 59.13718265287499,
 46.17315735879172,
 65.12727270428391,
 nan,
 43.562535226130485,
 53.608242425998235,
 54.55126652822712,
 82.25234136142836,
 72.81621831010278,
 47.55371296356379,
 49.84658685246301,
 46.88727534166849,
 67.9448010565946,
 80.40371714758415,
 44.0179630312083,
 55.03911560908212,
 67.061247266647,
 45.31649766595668,
 86.45215371186342,
 nan,
 40.6512210488641,
 46.10792924856668,
 52.18523602244639,
 46.796257733250904,
 92.68800585237666,
 392.7229338625503,
 57.03733053205078,
 41.42990717418455,
 38.72334079194784,
 nan,
 nan,
 49.02926453341669,
 52.54318284040338,
 690.3807795741332,
 100.16597731102988,
 49.7555817199899,
 57.25404444873436,
 47.45954060297256,
 43.805308058932596,
 50.1431737837583,
 83.24633171510443,
 61.88052658076779,
 70.17855343931808,
 42.574114947085846,
 97.8067026329749,
 65.13805263884632,
 4045.0342769448803,
 77.78654589571455,
 37.66614080892729,
 36.80712573086501,
 48.78894307550188,
 43.6946067315633

In [155]:
features_df['Fare_per_dist']=get_fare_per_distance(features_df)
features_df

Streaming output truncated to the last 5000 lines.
12176 60.81742180600344
12177 67.39382134261399
12178 45.38330704088775
12179 101765.64412107253
12180 48.78334937777761
12181 73750.93673465875
12182 47.990648156166905
12183 41.29111100130782
12184 49.21013261799155
12185 57.58719261542081
12186 42.62768973490465
12187 65.36958817988467
12188 40.66755203911348
12189 39.098560107104326
12190 37.577230753328955
12191 22.05001705953534
12192 67.98148444266417
12193 31.675481578863643
12194 101.43690231180352
12195 39.93312243759037
12196 56.2403354026749
12197 38.427755833554976
12198 44.73143381526923
12199 36.664438221279354
12200 44.008831732774304
12201 52.700868168215464
12202 47.05965439801223
12203 52.26556193888909
12204 47.94536996264846
12205 52.542152458845834
12206 42643.76211163702
12207 46.70267772993097
12208 38.96986890289784
12209 53.15906935147552
12210 39.16221864113386
12211 85.10590208510014
12212 60.856253541097594
12213 37.58487054088617
12214 55.75038478309266
12

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Fare_per_dist
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,51.017424
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,59.137183
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,46.173157
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,65.127273
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,86.605971
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,52.512899
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,62.839636
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,42.656035


# Remove wrong fare per dist

In [156]:
# def drop_wrong_fare_per_dist(df,label):
#   for index, row in df.iterrows():
#     if(row['Fare_per_dist']<0):
#       print(index)
#       df=df.drop(index)
#       label=label.drop(index)

  
#   return df,label


In [157]:
# features_df,labels_df=drop_wrong_fare_per_dist(features_df,labels_df)
# # features_df=features_df.drop(1)
# features_df

#Fare per sec

In [158]:
import math

def get_fare_per_min(df):
  fare_per_min=[]
  for index, row in df.iterrows():
    if(row['duration']==0):
     
      fare_per_min.append(0)
    else:
      fare_per_minute=(float(row['fare']))/(float(row['duration']))

      # if(math.isnan(fare_per_minute)):
      #   fare_per_minute=0
      
      fare_per_min.append(fare_per_minute)
   
        
    
  return fare_per_min




In [159]:
get_fare_per_min(features_df)

[0.32412470023980816,
 0.25012642225031606,
 0.2774977000919963,
 0.13762541806020068,
 nan,
 0.3125975931904902,
 0.21398073836276083,
 0.23859714928732184,
 0.2786666666666667,
 0.1675698505523067,
 26.70653846153846,
 7.599677419354839,
 0.22495773247140724,
 0.08826190476190476,
 0.3787164179104478,
 0.159410092395167,
 0.2073505434782609,
 0.3355985915492958,
 0.18942505133470225,
 0.25425364758698094,
 nan,
 0.21812027707808565,
 0.1578066378066378,
 0.27676470588235297,
 0.2918295218295218,
 0.5882122905027933,
 0.17656832298136646,
 0.17281553398058253,
 0.1559914712153518,
 0.3100173913043478,
 nan,
 nan,
 0.20241813602015113,
 0.21422909235037654,
 1.9087062937062937,
 0.24603351955307262,
 0.31243478260869567,
 0.22406976744186047,
 0.17665983606557376,
 0.2321146953405018,
 0.1939901650860555,
 0.2509597523219814,
 0.2484757505773672,
 0.32180851063829785,
 0.08464158977998582,
 0.07498430365296804,
 0.12458585858585859,
 0.041268758526603,
 0.23866941722537388,
 0.16981146

In [160]:
features_df['Fare_per_sec']=get_fare_per_min(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Fare_per_dist,Fare_per_sec
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,51.017424,0.324125
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,59.137183,0.250126
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,46.173157,0.277498
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,65.127273,0.137625
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,86.605971,0.236587
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,52.512899,0.270214
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,62.839636,0.289734
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,42.656035,0.155373


#sec per fare

# difference between fare_per_dist and fare_per_min

In [161]:


def get_fare_diff(df):
  fare_per_diff=[]
  for index, row in df.iterrows():
    if(row['Fare_per_dist']==0 or row['Fare_per_min']==0):
     
      fare_per_diff.append(0)
      # print('!!!!!!!')
    
    else:
      fare_diff=(float(row['Fare_per_dist']))/(float(row['Fare_per_min']))

      if(float(row['Fare_per_dist'])>float(row['Fare_per_min'])):
        fare_diff=fare_diff
        # print('aaaaaaaa')
      else:
        fare_diff=0-fare_diff
        # print('zzzzzzzz',index,fare_diff)
  
      fare_per_diff.append(fare_diff)
  
   
  # print(len(fare_per_diff))      
    
    
  
  return fare_per_diff


# get_fare_diff(features_df)

In [162]:
# features_df['Fare_diff']=get_fare_diff(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Fare_per_dist,Fare_per_sec
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,51.017424,0.324125
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,59.137183,0.250126
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,46.173157,0.277498
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,65.127273,0.137625
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,86.605971,0.236587
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,52.512899,0.270214
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,62.839636,0.289734
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,42.656035,0.155373


# pickup hour

In [163]:
# datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
def get_pickup_hour(df):
  pickup_hour=[]
 
  for index, row in df.iterrows():
    pickup_datetime=row['pickup_time']
    

    pickup_time=pickup_datetime.split(" ")[1];
    pickup_h=pickup_time.split(":")[0]
    pickup_hour.append(pickup_h)
   
    
   
    

    
  return pickup_hour

get_pickup_hour(features_df)

['0',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '6',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',

In [164]:
# features_df['pickup_hour']=get_pickup_hour(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Fare_per_dist,Fare_per_sec
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,51.017424,0.324125
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,59.137183,0.250126
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,46.173157,0.277498
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,65.127273,0.137625
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,86.605971,0.236587
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,52.512899,0.270214
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,62.839636,0.289734
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,42.656035,0.155373


# fare for only trip

In [165]:
# features_df['lbl']=labels_df['lb']

In [166]:
features_df.to_csv("features2.csv",index=True)

# Define featues

In [167]:
features=['additional_fare','duration','meter_waiting','meter_waiting_fare','fare','dist','Fare_per_dist','Fare_per_sec',]
num_features=['additional_fare','duration','meter_waiting','meter_waiting_fare','fare','dist','Fare_per_dist','Fare_per_sec']
# cat_features=['Time_type']


In [168]:
final_features_df=features_df[features]
labels_df
final_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,dist,Fare_per_dist,Fare_per_sec
0,10.5,834.0,56.0,0.0000,270.32,5.092770,51.017424,0.324125
1,10.5,791.0,47.0,0.0000,197.85,3.168058,59.137183,0.250126
2,10.5,1087.0,80.0,0.0000,301.64,6.305395,46.173157,0.277498
3,10.5,598.0,271.0,15.6638,82.30,0.861946,65.127273,0.137625
4,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,198.26,2.105376,86.605971,0.236587
17172,10.5,2151.0,428.0,0.0000,581.23,10.868377,52.512899,0.270214
17173,10.5,263.0,9.0,0.0000,76.20,1.045518,62.839636,0.289734
17174,10.5,858.0,115.0,0.0000,133.31,2.879077,42.656035,0.155373


In [169]:
# features_df.to_csv("s.csv",index=True)
final_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,dist,Fare_per_dist,Fare_per_sec
0,10.5,834.0,56.0,0.0000,270.32,5.092770,51.017424,0.324125
1,10.5,791.0,47.0,0.0000,197.85,3.168058,59.137183,0.250126
2,10.5,1087.0,80.0,0.0000,301.64,6.305395,46.173157,0.277498
3,10.5,598.0,271.0,15.6638,82.30,0.861946,65.127273,0.137625
4,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,198.26,2.105376,86.605971,0.236587
17172,10.5,2151.0,428.0,0.0000,581.23,10.868377,52.512899,0.270214
17173,10.5,263.0,9.0,0.0000,76.20,1.045518,62.839636,0.289734
17174,10.5,858.0,115.0,0.0000,133.31,2.879077,42.656035,0.155373


#Preprocess Pipeline

In [170]:
# final_features_df['fare'].fillna(0,inplace=True)
# final_features_df['additional_fare'].fillna(0,inplace=True)
# final_features_df

In [171]:
preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy="mean"))
])

cat_preprocessing_steps=Pipeline([
  ('cat_one_hot',OneHotEncoder())
])
preprocessor=ColumnTransformer(
    transformers = [
        ("features", preprocessing_steps, num_features)
        
    ],
    remainder="passthrough"
   
)
d=pd.DataFrame(preprocessor.fit_transform(final_features_df))
# d[0].std()
d

,0,1,2,3,4,5,6,7
0,-3.253376e-02,-1.070020e-01,-7.106395e-02,-6.886545e-02,-0.069913,0.005440,-6.781763e-02,-1.178256e-02
1,-3.253376e-02,-1.122972e-01,-7.218006e-02,-6.886545e-02,-0.189631,-0.039301,-6.758590e-02,-2.681070e-02
2,-3.253376e-02,-7.584681e-02,-6.808766e-02,-6.886545e-02,-0.018174,0.033627,-6.795589e-02,-2.125193e-02
3,-3.253376e-02,-1.360638e-01,-4.440135e-02,-3.521680e-02,-0.380515,-0.092907,-6.741494e-02,-4.965827e-02
4,-1.511175e-17,3.349444e-18,-6.280208e-18,-1.632854e-17,0.075576,0.076454,-8.795401e-18,-2.094143e-18
...,...,...,...,...,...,...,...,...
17167,-3.253376e-02,-1.065095e-01,-6.647550e-02,-5.721824e-02,-0.188954,-0.064003,-6.680194e-02,-2.956037e-02
17168,-3.253376e-02,5.517756e-02,-2.493145e-02,-6.886545e-02,0.443699,0.139695,-6.777495e-02,-2.273119e-02
17169,-3.253376e-02,-1.773168e-01,-7.689252e-02,-6.886545e-02,-0.390592,-0.088640,-6.748023e-02,-1.876692e-02
17170,-3.253376e-02,-1.040466e-01,-6.374724e-02,-6.886545e-02,-0.296249,-0.046018,-6.805627e-02,-4.605396e-02


# model define

In [172]:
# estimator=RandomForestClassifier(bootstrap= False,max_depth= 20,max_features= 'auto',min_samples_leaf= 4,min_samples_split= 2,n_estimators= 754)
estimator=XGBClassifier(learning_rate=0.013000000000000001,max_depth=50,n_estimators=366)

# estimator=XGBClassifier()
estimator
# rf_random.best_params_


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.013000000000000001, max_delta_step=0,
              max_depth=50, min_child_weight=1, missing=None, n_estimators=366,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

#Full pipeline define

In [173]:
full_Pipeline=Pipeline([
  ("preprocess",preprocessor),
  ("estimator",estimator)
])
labels_df
final_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,dist,Fare_per_dist,Fare_per_sec
0,10.5,834.0,56.0,0.0000,270.32,5.092770,51.017424,0.324125
1,10.5,791.0,47.0,0.0000,197.85,3.168058,59.137183,0.250126
2,10.5,1087.0,80.0,0.0000,301.64,6.305395,46.173157,0.277498
3,10.5,598.0,271.0,15.6638,82.30,0.861946,65.127273,0.137625
4,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,198.26,2.105376,86.605971,0.236587
17172,10.5,2151.0,428.0,0.0000,581.23,10.868377,52.512899,0.270214
17173,10.5,263.0,9.0,0.0000,76.20,1.045518,62.839636,0.289734
17174,10.5,858.0,115.0,0.0000,133.31,2.879077,42.656035,0.155373


In [174]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 700, num = 8)]

max_depth = [int(x) for x in np.linspace(5,80, num = 10)]
# max_depth.append(None)

learning_rate=[float(x) for x in np.linspace(0.002, 0.02, num = 10)]



# Create the random grid
random_grid = {'estimator__n_estimators': n_estimators,
               'estimator__learning_rate': learning_rate,
               'estimator__max_depth': max_depth}
pprint(random_grid)

{'estimator__learning_rate': [0.002,
                              0.004,
                              0.006,
                              0.008,
                              0.01,
                              0.012,
                              0.014,
                              0.016,
                              0.018000000000000002,
                              0.02],
 'estimator__max_depth': [5, 13, 21, 30, 38, 46, 55, 63, 71, 80],
 'estimator__n_estimators': [100, 185, 271, 357, 442, 528, 614, 700]}


In [175]:
# from sklearn.model_selection import GridSearchCV


# rf_random = GridSearchCV (estimator=estimator,param_grid=random_grid,cv=6,scoring='f1',verbose=4,return_train_score=True)

# rf_random.get_params()

In [176]:
from sklearn.model_selection import RandomizedSearchCV


rf_random = RandomizedSearchCV(estimator = full_Pipeline, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42)

rf_random.get_params()

{'cv': 3, 'error_score': nan, 'estimator': Pipeline(memory=None,
          steps=[('preprocess',
                  ColumnTransformer(n_jobs=None, remainder='passthrough',
                                    sparse_threshold=0.3,
                                    transformer_weights=None,
                                    transformers=[('features',
                                                   Pipeline(memory=None,
                                                            steps=[('standard_scaler',
                                                                    StandardScaler(copy=True,
                                                                                   with_mean=True,
                                                                                   with_std=True)),
                                                                   ('simple_imputer',
                                                                    SimpleImputer(add_indicator=False,
    

#split dataset

In [177]:
X_train, X_eval, y_train, y_eval = train_test_split(
    final_features_df,
    labels_df,
    test_size=0.2,
    shuffle=True,
    random_state=6
)
final_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,dist,Fare_per_dist,Fare_per_sec
0,10.5,834.0,56.0,0.0000,270.32,5.092770,51.017424,0.324125
1,10.5,791.0,47.0,0.0000,197.85,3.168058,59.137183,0.250126
2,10.5,1087.0,80.0,0.0000,301.64,6.305395,46.173157,0.277498
3,10.5,598.0,271.0,15.6638,82.30,0.861946,65.127273,0.137625
4,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,198.26,2.105376,86.605971,0.236587
17172,10.5,2151.0,428.0,0.0000,581.23,10.868377,52.512899,0.270214
17173,10.5,263.0,9.0,0.0000,76.20,1.045518,62.839636,0.289734
17174,10.5,858.0,115.0,0.0000,133.31,2.879077,42.656035,0.155373


# train model

In [178]:
np.testing.assert_array_equal(final_features_df.index.values,labels_df.index.values)
# payments_df


In [179]:
# rf_random.fit(X_train,y_train['lb'])
full_Pipeline.fit(X_train,y_train['lb'])


Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('features',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                           

In [180]:
# rf_random.best_params_

# full_Pipeline.score(X_eval,y_eval['lb'])

In [181]:
y_pred=full_Pipeline.predict(X_eval)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [182]:
full_Pipeline.score(X_eval,y_eval['lb'])

0.9513828238719069

In [183]:
fbeta_score(y_eval,y_pred,beta=0.5)

0.966067358825019

In [184]:
metrics.SCORERS.keys()
features

['additional_fare',
 'duration',
 'meter_waiting',
 'meter_waiting_fare',
 'fare',
 'dist',
 'Fare_per_dist',
 'Fare_per_sec']

In [185]:
from sklearn.model_selection import cross_val_score
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(full_Pipeline, final_features_df, labels_df['lb'], cv=kfold, scoring='f1')
cv_results

array([0.9682136 , 0.97612225, 0.97027804, 0.97290405, 0.97669965,
       0.97264631, 0.97925311, 0.97104677, 0.97607656, 0.97075652])

In [186]:
cv_results.mean()

0.9733996864394847

In [187]:
f1_score(y_eval, y_pred)

0.973369478552065

# train for full dataset

In [188]:
features

['additional_fare',
 'duration',
 'meter_waiting',
 'meter_waiting_fare',
 'fare',
 'dist',
 'Fare_per_dist',
 'Fare_per_sec']

In [189]:
# predict=estimator.predict(preprocessed_test_features);
full_Pipeline.fit(final_features_df,labels_df['lb'])

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('features',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                           

In [190]:
test_df['dist']=getDistance(test_df)
# test_df['Time_type']=get_time_type(test_df)
test_df['Fare_per_dist']=get_fare_per_distance(test_df)
test_df['Fare_per_sec']=get_fare_per_min(test_df)
# test_df['Fare_diff']=get_fare_diff(test_df)
# test_df['pickup_hour']=get_pickup_hour(test_df)
test_df[features]

Streaming output truncated to the last 5000 lines.
3577 45.348936942252976
3578 55.51078436579875
3579 39.13032912453974
3580 108.2833883592472
3581 40.66935391647253
3582 8.20032834022238
3583 53.57507346550166
3584 105.75540462368555
3585 64.5801372144427
3586 38.60015695028504
3587 6995.368376024665
3588 34.80347124679397
3589 45.371083748253184
3590 36.71403477573933
3591 121.80110854625937
3592 48.31373464361165
3593 49.15131543000564
3594 185.81113922917103
3595 42.356544386369585
3596 40.674122851335355
3597 45.55363846034234
3598 40.90834744802753
3599 80.73008493621909
3600 47.63812723604219
3601 45.9007285825862
3602 79.00658071441366
3603 39.27242867305376
3604 44287.72200955263
3605 47.68742916894834
3606 53.4414275819812
3607 61.76320251406254
3608 46.95354195099319
3609 43.45984735336903
3610 72.79092306085752
3611 40.0995151886478
3612 63.728707890347025
3613 49.74274837131737
3614 52.783306657719024
3615 114.74417493701841
3616 89.45022890904495
3617 3360.684134624374
3

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,dist,Fare_per_dist,Fare_per_sec
0,10.5,924,42,2.44860,289.27,6.705702,41.206933,0.313063
1,10.5,4249,20,0.00000,1912.70,41.558513,45.771609,0.450153
2,10.5,1552,255,2.65880,394.00,5.916678,64.367402,0.253866
3,10.5,462,16,0.00000,154.32,3.301761,43.558571,0.334026
4,10.5,814,392,12.36920,147.47,2.588542,48.135517,0.181167
...,...,...,...,...,...,...,...,...
8571,10.5,1723,429,24.83332,388.48,3.934272,89.761643,0.225467
8572,10.5,1378,80,0.00000,379.85,7.517433,49.132466,0.275653
8573,10.5,418,56,3.28440,112.79,2.057225,48.125809,0.269833
8574,10.5,1604,548,31.67440,248.46,3.900888,52.881696,0.154900


In [191]:
# # prepros_test=pd.DataFrame(preprocessor.fit_transform(test_df))
# # prepros_test
# test_df['fare'].fillna(0,inplace=True)
# test_df['additional_fare'].fillna(0,inplace=True)
# test_df[features]

In [192]:
t=full_Pipeline.predict(test_df[features])
t

array([1, 1, 1, ..., 1, 1, 1])

In [193]:
submission_df['prediction']=t

In [194]:
submission_df

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
...,...,...
8571,222856243,1
8572,222857785,1
8573,222858416,1
8574,222858691,1


In [195]:
submission_df.to_csv("submission_29.csv",index=True)